In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from utils import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colormaps
from matplotlib.animation import FuncAnimation
from scipy.stats import uniform
from observision_models import NormalObservation
from IPython.display import HTML
from matplotlib.animation import ImageMagickWriter

from ball_throwing_simulator import BallThrowingSimulator

# Simulate ball throwing and generate Observations

In [ ]:
ball_num = 1
init_state = np.array([[0], [50], [300], [300]])  # Initial state: [x, y, vx, vy]
simulator = BallThrowingSimulator(delta_t=0.5, init_state=init_state, ball_num=ball_num, observation_variance=[10, 10])
# Simulate the motion and observations
states, observations = simulator.simulate(time=30)
print(states.shape, observations.shape)
# Show the simulated states and observations
fig, ax = plt.subplots(figsize=(12, 12))
plot_observations(ax, states, observations)

# Initilaize
## State
$$[X, Y, V_x, V_y]^T$$

## Uniform Random Particle Sets
$$X, Y \in [0, 3000]$$
$$V_x, V_y \in [0, 200]$$

In [ ]:
from partickle_filter import ParticleFilter
pf = ParticleFilter(5000)

init_particles, init_weights = pf.init_particles, pf.init_weights

# show initial particles
fig, ax = plt.subplots(figsize=(12, 12))
# ax.set_xlim(0, 3000)
# ax.set_ylim(0, 3000)
ax.set_title('Initial Particles')
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')
plot_particles(ax, init_particles, init_weights)
plot_observations(ax, states, observations, NormalObservation.R)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
from IPython.display import clear_output, display
import time

# 假设已有变量: states, observations, pf（含 pf.snaps、pf.update）

# 输出目录
os.makedirs("outputs", exist_ok=True)

# 初始化画布
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_title("Particle Filter Animation")
all_x = np.concatenate((states[:, 0], observations[:, 0]))
all_y = np.concatenate((states[:, 1], observations[:, 1]))
x_min, x_max = np.min(all_x) - 1000, np.max(all_x) + 200
y_min, y_max = -np.max(all_y), 1.5 * np.max(all_y)
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_aspect('equal')


# 初始化粒子状态
particles, weights = pf.init_particles, pf.init_weights

# 初始化绘图对象
# particles_scatter = ax.scatter(particles[:,0], particles[:, 1], c='blue', alpha=0.7)
particles_quiver = ax.quiver(particles[:,0], particles[:, 1], 
                             particles[:,2], particles[:, 3], 
                             color = plt.cm.viridis(weights),
                             angles='xy', scale_units='xy', scale=1)
observations_plot, = ax.plot([], [], 'rx', label='Observations')
true_state_plot, = ax.plot(states[0, 0], states[0, 1], 'g--', label='True State')

plt.legend()

plt.savefig(f"outputs/frame_000.png", dpi=150)
display(fig)
# 主循环：逐帧绘图并保存为 PNG
for frame in range(1, states.shape[0]):
    particles, weights = pf.update(particles, weights, observations[frame-1])

    # particles_scatter.set_offsets(particles[:, :2])
    # particles_scatter.set_color(plt.cm.viridis(weights))

    particles_quiver.set_offsets(particles[:, :2])
    particles_quiver.set_UVC(particles[:, 2], particles[:, 3])
    particles_quiver.set_color(plt.cm.viridis(weights))

    observations_plot.set_data(observations[:frame, 0], observations[:frame, 1])
    true_state_plot.set_data(states[:frame, 0], states[:frame, 1])

    clear_output(wait=True)
    display(fig)
    # 保存帧
    plt.savefig(f"outputs/frame_{frame:03d}.png", dpi=150)

# plt.close(fig)  # 完成后关闭图形
plt.ioff()
plt.close(fig)

# save to gif
gif_frames = [Image.open(f"outputs/frame_{frame:03d}.png") for frame in range(states.shape[0])]
gif_path = "outputs/particle_filter_animation.gif"
gif_frames[0].save(gif_path, save_all=True, append_images=gif_frames[1:], duration=300, loop=0)
